In [1]:
from pyopenms import *
import os
import glob
import pandas as pd

ModuleNotFoundError: No module named 'pyopenms'

In [ ]:
# constant path for interim files
path = "results/interim"
if not os.path.exists(path): # if it doesn't exist
    os.mkdir("results") # create a results directory
    os.mkdir(path)  # create an interim directory for temporary results

In [ ]:
# 1) Feature Detection

input_mzml_files = glob.glob('tests/Euphorbia/Targeted/toppas_input/*.mzML') # introduce a set of mzML files from the Example_data directory

# 1.1) Mass trace detection

#mtd
mass_error_ppm = 10.0
noise_threshold_int = 1e6
chrom_fwhm = 10.0
reestimate_mt_sd = 'true'
quant_method = 'max_height'
trace_termination_criterion = 'outlier'
trace_termination_outliers = 3
min_sample_rate = 0.3
min_trace_length = 1.0
max_trace_length = 100.0

#epd
enabled = 'true'
width_filtering = 'fixed'
min_fwhm = 0.0
max_fwhm = 30.0
masstrace_snr_filtering = 'false'

#ffm
local_rt_range = 7.0
local_mz_range = 6.0
charge_lower_bound = 1
charge_upper_bound = 3
report_summed_ints = 'false'
enable_RT_filtering = 'false'
isotope_filtering_model = 'none'
mz_scoring_13C = 'false'
use_smoothed_intensities = 'true'
report_convex_hulls = 'false'
remove_single_traces = 'false'
mz_scoring_by_elements = 'false'
elements = 'CHNOPS'


width_filtering
for filename in input_mzml_files: # for each file in the set of files
    print("Mass Trace Detection: ", filename) #print the filename
    exp = MSExperiment()    
    MzMLFile().load(filename, exp) # load each mzML file to an OpenMS file format (MSExperiment)

    mass_traces = [] # introduce an empty list where the mass traces will be loaded
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults() # get the default parameters in order to edit them
    mtd_par.setValue("mass_error_ppm", mass_error_ppm) # high-res instrument, orbitraps
    mtd_par.setValue("noise_threshold_int", noise_threshold_int) # data-dependent (usually works for orbitraps)
    mtd_par.setValue("chrom_fwhm", chrom_fwhm)
    mtd_par.setValue("reestimate_mt_sd", reestimate_mt_sd)
    mtd_par.setValue("quant_method", quant_method)
    mtd_par.setValue("trace_termination_criterion", trace_termination_criterion)
    mtd_par.setValue("trace_termination_outliers", trace_termination_outliers)
    mtd_par.setValue("min_sample_rate", min_sample_rate)
    mtd_par.setValue("min_trace_length", min_trace_length)
    mtd_par.setValue("max_trace_length", max_trace_length)
    mtd.setParameters(mtd_par) # set the new parameters
    mtd.run(exp, mass_traces, 0) # run mass trace detection

# 1.2) Elution peak detection

    print("Elution Peak Detection: ", filename)
    mass_traces_deconvol = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", width_filtering) # The fixed setting filters out mass traces outside the [min_fwhm: 1.0, max_fwhm: 60.0] interval
    epd_par.setValue("min_fwhm", min_fwhm) 
    epd_par.setValue("max_fwhm", max_fwhm) 
    epd_par.setValue("enabled", enabled) 
    epd_par.setValue("masstrace_snr_filtering", masstrace_snr_filtering) 
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_deconvol)
     
# 1.3) Feature detection

    print("Feature Detection: ", filename)
    feature_map_FFM = FeatureMap() # output features 
    chrom_out = [] # output chromatograms 
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("remove_single_traces", remove_single_traces) # remove mass traces without satellite isotopic traces
    ffm_par.setValue("local_rt_range", local_rt_range)  
    ffm_par.setValue("charge_lower_bound", charge_lower_bound)      
    ffm_par.setValue("charge_upper_bound", charge_upper_bound)  
    ffm_par.setValue("report_summed_ints", report_summed_ints)      
    ffm_par.setValue("enable_RT_filtering", enable_RT_filtering)      
    ffm_par.setValue("isotope_filtering_model", isotope_filtering_model)      
    ffm_par.setValue("mz_scoring_13C", mz_scoring_13C)    
    ffm_par.setValue("use_smoothed_intensities", use_smoothed_intensities)      
    ffm_par.setValue("report_convex_hulls", report_convex_hulls)  
    ffm_par.setValue("mz_scoring_by_elements", mz_scoring_by_elements)      
    ffm_par.setValue("elements", elements)       
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_deconvol, feature_map_FFM, chrom_out)
    feature_map_FFM.setUniqueIds() # Assigns a new, valid unique id per feature
    feature_map_FFM.setPrimaryMSRunPath([filename.encode()]) # Sets the file path to the primary MS run (usually the mzML file)
    FeatureXMLFile().store(os.path.join(path, os.path.basename(filename)[:-5] + ".featureXML"), feature_map_FFM)
    
print("Finished Feature Detection")

In [ ]:
# load feature files 

input_feature_files = glob.glob('results/interim/*.featureXML') # set of feature files

feature_maps = [] # empty list to fill with FeatureMaps: the OpenMS file format for feature files
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap) # load each file to a feature map
    feature_maps.append(fmap) # append all maps to the empty list 

In [ ]:
# 4) Feature grouping

feature_grouper = FeatureGroupingAlgorithmKD()

consensus_map = ConsensusMap()
file_descriptions = consensus_map.getColumnHeaders()

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, ColumnHeader())
    file_description.filename = os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())
    file_description.size = feature_map.size()
    file_descriptions[i] = file_description

keep_subelements = 'true'
mz_unit = 'ppm'
nr_partitions = 50
warp_enabled = 'false'
warp_rt_tol = 30.0
warp_mz_tol = 10.0
warp_max_pairwise_log_fc = 0.0
warp_min_rel_cc_size = 0.0
warp_max_nr_conflicts = -1
link_rt_tol = 30
link_mz_tol = 10
link_charge_merging = 'With_charge_zero'
link_adduct_merging = 'Any'
distance_RT_exponent = 1.0
distance_MZ_weight = 2.0
distance_intensity_log_transform = 1.0
distance_intensity_weight = 1.0
LOWESS_span = 0.0
LOWESS_num_iterations = 3
LOWESS_delta = -1.0
LOWESS_interpolation_type = 'cspline'
LOWESS_extrapolation_type = 'four-point-linear'

feature_grouper_par = feature_grouper.getDefaults()
feature_grouper_par.setValue("keep_subelements", keep_subelements)
feature_grouper_par.setValue("mz_unit", mz_unit) 
feature_grouper_par.setValue("nr_partitions", nr_partitions) 
feature_grouper_par.setValue("warp:enabled", warp_enabled) 
feature_grouper_par.setValue("warp:rt_tol", warp_rt_tol) 
feature_grouper_par.setValue("warp:mz_tol", warp_mz_tol) 
feature_grouper_par.setValue('warp:max_pairwise_log_fc', warp_max_pairwise_log_fc) 
feature_grouper_par.setValue("warp:min_rel_cc_size", warp_min_rel_cc_size) 
feature_grouper_par.setValue("warp:max_nr_conflicts", warp_max_nr_conflicts) 
feature_grouper_par.setValue("link:rt_tol", link_rt_tol)
feature_grouper_par.setValue("link:mz_tol", link_mz_tol) 
feature_grouper_par.setValue("link:adduct_merging", link_adduct_merging) 
feature_grouper_par.setValue("link:charge_merging", link_charge_merging) 
feature_grouper_par.setValue("distance_RT:exponent", distance_RT_exponent) 
feature_grouper_par.setValue("distance_MZ:weight", distance_MZ_weight)
feature_grouper_par.setValue("distance_intensity:log_transform", distance_intensity_log_transform) 
feature_grouper_par.setValue("distance_intensity:weight", distance_intensity_weight) 
feature_grouper_par.setValue("LOWESS:span", LOWESS_span) 
feature_grouper_par.setValue("LOWESS:num_iterations", LOWESS_num_iterations)
feature_grouper_par.setValue("LOWESS:delta", LOWESS_delta)
feature_grouper_par.setValue("LOWESS:num_iterations", LOWESS_num_iterations)
feature_grouper_par.setValue("LOWESS:interpolation_type", LOWESS_interpolation_type)
feature_grouper_par.setValue("LOWESS:extrapolation_type", LOWESS_extrapolation_type)

#print(feature_grouper_par)

feature_grouper.group(feature_maps, consensus_map)
consensus_map.setUniqueIds()
consensus_map.setColumnHeaders(file_descriptions)


Consensus_file = os.path.join(path, 'consensus' + ".consensusXML")
ConsensusXMLFile().store(Consensus_file, consensus_map)

df = consensus_map.get_df()
df= df.drop(columns="sequence")
df

In [ ]:
fig = px.scatter(df[df["quality"] > 0.01], x="RT", y="mz", color="quality")
fig.update_layout(title="Consensus features")
fig.show()

In [ ]:
# Export a .TXT table of features 

output_file = "results/GNPSexport/FeatureQuantificationTable.txt"
IonIdentityMolecularNetworking.writeFeatureQuantificationTable(cmap, output_file)

In [ ]:
# Create a metadata table from the list of mzML files compatible for GNPS

metadata = pd.DataFrame()
metadata["filename"] = [file for file in os.listdir("Example_data") if file.endswith(".mzML")]
metadata["ATTRIBUTE_MAPID"]= ["MAP" + str(i) for i in range(len(metadata))]
metadata['ATTRIBUTE_compound'] = metadata['filename'].replace(".mzML", value="", regex=True)
metadata.to_csv("results/GNPSexport/metadata.tsv", sep='\t')

metadata